In [8]:
from bs4 import BeautifulSoup
import pandas as pd
import pickle
import random
import requests
from selenium import webdriver
#from selenium.webdriver.common.keys import Keys
import time

In [10]:
df_headers = ["address", "zip code", "unit", "rent", "rooms", "beds", "baths", "studio", "link"]
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [52]:
#function to get number of rooms, beds, baths, and whether there is a studio
def parse_details(details_list, num_rooms = -1, num_beds = -1, num_baths = -1, has_studio = 0):
    for d in details_list:
        #print("parsing: " + d.text)
        if len(d.text) == 0:
            break
        elif d.text == "studio":
            num_beds = 0
            has_studio = 1
        else:
            keyword = (d.text.split()[1])

            if keyword in ["room", "rooms"]:
                count = float(d.text.split()[0].replace('+',''))
                num_rooms = count

            elif keyword in ["bed", "beds"]:
                count = float(d.text.split()[0].replace('+',''))
                num_beds = count
                
            elif keyword in ["bath", "baths"]:
                count = float(d.text.split()[0].replace('+',''))
                num_baths = count
    return([num_rooms, num_beds, num_baths, has_studio])

In [11]:
apt_data = []

In [17]:
def scrape_page(listings):
    for l in listings:
        driver.get(l)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        address_raw = soup.find(class_ = "backend_data BuildingInfo-item").find_all(class_ = "Text")
        address = address_raw[0].text.strip() + ", " + address_raw[1].text.strip()
        street_address = address.split(",")[0]
        unit = soup.find(class_ = "building-title").a.text.split()[-1]
        zip_code = int(address.split()[-1])
        price = soup.find(class_ = "price").text.split()[0]
        all_details = soup.find_all(class_ = "detail_cell")
        [num_rooms, num_beds, num_baths, has_studio] = parse_details(all_details)
        apt_dict = dict(zip(df_headers, [street_address, zip_code, unit, price, num_rooms, num_beds, num_baths, has_studio, l]))
        apt_data.append(apt_dict)
        #sq_ft = num_rooms = num_beds = num_baths = -1
        time.sleep(random.randint(1, 4)) # waiting 5 seconds every time triggered a captcha quickly, but this doesn't

In [42]:
driver = webdriver.Chrome(chromedriver)
listings = []

for page_num in range(1, 11):
    scraping_url = f"https://streeteasy.com/for-rent/brooklyn?page={page_num}"
    driver.get(scraping_url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    all_urls = soup.find_all('a', class_ = "listingCard-globalLink jsGlobalListingCardLink", attrs = "href")
    for u in all_urls:
        link = u.get("href")
        listings.append(link)
    time.sleep(random.randint(5, 10))
        
driver.quit()

In [43]:
print(listings)

['https://streeteasy.com/building/570-nostrand-avenue-brooklyn/6e', 'https://streeteasy.com/building/570-nostrand-avenue-brooklyn/5e', 'https://streeteasy.com/building/570-nostrand-avenue-brooklyn/4e', 'https://streeteasy.com/building/570-nostrand-avenue-brooklyn/6b', 'https://streeteasy.com/building/570-nostrand-avenue-brooklyn/6d', 'https://streeteasy.com/building/570-nostrand-avenue-brooklyn/5d', 'https://streeteasy.com/building/225-suydam-street-brooklyn/2r', 'https://streeteasy.com/building/231-jefferson-avenue-brooklyn/2r', 'https://streeteasy.com/building/plg-123-linden-boulevard-brooklyn/13s', 'https://streeteasy.com/building/225-suydam-street-brooklyn/2l', 'https://streeteasy.com/building/279-st-james-place-brooklyn/3f', 'https://streeteasy.com/building/1711-63-street-brooklyn/walk-in', 'https://streeteasy.com/building/1-flatbush/1806', 'https://streeteasy.com/building/1-flatbush/1106', 'https://streeteasy.com/building/1212-lincoln-place-brooklyn/rental/3511520', 'https://stre

In [58]:
apt_data = []
driver = webdriver.Chrome(chromedriver)
scrape_page(listings)
driver.quit()

In [59]:
apt_frame = pd.DataFrame.from_dict(apt_data)
apt_frame

,address,zip code,unit,rent,rooms,beds,baths,studio,link
0,570 Nostrand Avenue,11216,#6E,"$2,375",2.0,1.0,1.0,0,https://streeteasy.com/building/570-nostrand-a...
1,570 Nostrand Avenue,11216,#5E,"$2,350",2.0,1.0,1.0,0,https://streeteasy.com/building/570-nostrand-a...
2,570 Nostrand Avenue,11216,#4E,"$2,350",2.0,1.0,1.0,0,https://streeteasy.com/building/570-nostrand-a...
3,570 Nostrand Avenue,11216,#6B,"$3,100",3.0,2.0,2.0,0,https://streeteasy.com/building/570-nostrand-a...
4,570 Nostrand Avenue,11216,#6D,"$3,200",3.0,2.0,2.0,0,https://streeteasy.com/building/570-nostrand-a...
...,...,...,...,...,...,...,...,...,...
105,439 Knickerbocker Avenue,11237,#3D,"$2,200",5.0,2.0,1.0,0,https://streeteasy.com/building/439-knickerboc...
106,One South First 1 South 1st Street,11249,#34N,"$3,625",2.0,0.0,1.0,1,https://streeteasy.com/building/one-south-firs...
107,573 East 22nd Street,11226,#GARDEN,"$2,745",8.0,4.0,1.5,0,https://streeteasy.com/building/573-east-22-st...
108,429 43rd Street,11232,#2F,"$2,500",6.0,3.0,2.0,0,https://streeteasy.com/building/429-43-street-...


In [61]:
#pickling for regression notebook
apt_frame.to_pickle("data/MVP_data.pickle")